In [1]:
suppressPackageStartupMessages({
    library(org.Hs.eg.db)
    library(org.Mm.eg.db)
    library(org.Pvitticeps.eg.db)
    library(org.Pmarinus.eg.db)
    library(org.Bfloridae.eg.db)
    library(clusterProfiler)
    library(enrichplot)
    library(ggplot2)
    library(igraph)
})

Warning message:
“package ‘Biobase’ was built under R version 4.3.3”
Warning message:
“package ‘IRanges’ was built under R version 4.3.3”
Warning message:
“package ‘S4Vectors’ was built under R version 4.3.3”


In [2]:
# human and mouse packages were downloaded and the background is default rather than only protein coding genes
# read protein coding gene IDs as background for all species
Hsap_bg <- read.table("Hsap_bg")$V1
Mmus_bg <- read.table("Mmus_bg")$V1
Pvit_bg <- read.table("Pvit_bg")$V1
Pmar_bg <- read.table("Pmar_bg")$V1
Bflo_bg <- read.table("../0bin/Bflo_T1_background_genes.txt")$V1

In [3]:
species <- c("Hsap","Mmus","Pvit","Pmar","Bflo")
path <- "/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/03.paralog_inferring/"
for (s in species){
    file_ <- paste0(path, s, ".paralogs.txt")
    assign(x = paste0(s,".para"), value = read.table(file_, header = T))
}

In [4]:
species <- c("Hsap","Mmus","Pvit","Pmar")
path <- "/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/03.ohnolog_identification/Ohnologs-v2.0/7_FilterOhnologs/"
for (s in species){
    file_ <- paste0(path, s, ".ohnologs.txt")
    assign(x = paste0(s,".ohno"), value =  read.table(file_, header = T))
}

In [5]:
species <- c("Hsap","Mmus","Pvit","Pmar")
for (s in species){
    tmp1 <- get(paste0(s, ".para"))
    tmp2 <- get(paste0(s, ".ohno"))
    
    SSDparalogs <- tmp1[!tmp1$Dup1 %in% unique(c(tmp2$Ohno1, tmp2$Ohno2)),]
    SSDparalogs <- SSDparalogs[!SSDparalogs$Dup2 %in% unique(c(tmp2$Ohno1, tmp2$Ohno2)),]
    assign(x = paste0(s, ".SSDparalogs"), value = SSDparalogs)
    
}

In [16]:
# GO for human
s = "Hsap"
p <- c("ohno", "SSDparalogs")
for (c in p){
    tmp <- get(paste0(s,".",c))[,1:2]
    colnames(tmp) <- c("Dup1", "Dup2")
    
    ego2 <- enrichGO(gene         = unique(c(tmp$Dup1, tmp$Dup2)),
                 OrgDb         = org.Hs.eg.db,
                 keyType = "ENSEMBL",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.01,
                 qvalueCutoff  = 0.05,
                 universe = Hsap_bg,
                 ont  = "BP")
    ego2 <- clusterProfiler::simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)
    assign(x = paste0(s, '.', c, '.ego2'), ego2)
    
    fig <- dotplot(ego2, showCategory=30)
    ggsave(filename = paste0(s,"_",c,".GO.pdf"), fig, width = 6, height = 12)
}

In [17]:
# GO for mouse
s = "Mmus"
p <- c("ohno", "SSDparalogs")
for (c in p){
    tmp <- get(paste0(s,".",c))[,1:2]
    colnames(tmp) <- c("Dup1", "Dup2")
    
    ego2 <- enrichGO(gene         = unique(c(tmp$Dup1, tmp$Dup2)),
                 OrgDb         = org.Mm.eg.db,
                 keyType = "SYMBOL",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.01,
                 qvalueCutoff  = 0.05,
                 universe = Mmus_bg,
                 ont  = "BP")
    ego2 <- clusterProfiler::simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)
    assign(x = paste0(s, '.', c, '.ego2'), ego2)
    fig <- dotplot(ego2, showCategory=30)
    ggsave(filename = paste0(s,"_",c,".GO.pdf"), fig, width = 6, height = 12)
}

In [18]:
# GO for lizard
s = "Pvit"
p <- c("ohno", "SSDparalogs")
for (c in p){
    tmp <- get(paste0(s,".",c))[,1:2]
    colnames(tmp) <- c("Dup1", "Dup2")
    
    ego2 <- enrichGO(gene         = unique(c(tmp$Dup1, tmp$Dup2)),
                 OrgDb         = org.Pvitticeps.eg.db,
                 keyType = "GID",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.01,
                 qvalueCutoff  = 0.05,
                 universe = Pvit_bg,
                 ont  = "BP")
    ego2 <- clusterProfiler::simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)
    assign(x = paste0(s, '.', c, '.ego2'), ego2)
    
    fig <- dotplot(ego2, showCategory=30)
    ggsave(filename = paste0(s,"_",c,".GO.pdf"), fig, width = 6, height = 12)
}

In [19]:
# GO for lamprey
s = "Pmar"
p <- c("ohno", "SSDparalogs")
for (c in p){
    tmp <- get(paste0(s,".",c))[,1:2]
    colnames(tmp) <- c("Dup1", "Dup2")
    genes <- unique(c(tmp$Dup1, tmp$Dup2))
    genes <- gsub('_','-', genes)
    ego2 <- enrichGO(gene         = genes,
                 OrgDb         = org.Pmarinus.eg.db,
                 keyType = "GID",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.01,
                 qvalueCutoff  = 0.1,
                 universe = Pmar_bg,
                 ont  = "BP")
    ego2 <- clusterProfiler::simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)
    assign(x = paste0(s, '.', c, '.ego2'), ego2)
    
    fig <- dotplot(ego2, showCategory=30)
    ggsave(filename = paste0(s,"_",c,".GO.pdf"), fig, width = 6, height = 12)
}

In [20]:
# GO for Amphioxus
s = "Bflo"
p <- c("para")
for (c in p){
    tmp <- get(paste0(s,".",c))[,1:2]
    colnames(tmp) <- c("Dup1", "Dup2")
    
    ego2 <- enrichGO(gene         = gsub("_","-",unique(c(tmp$Dup1, tmp$Dup2))),
                 OrgDb         = org.Bfloridae.eg.db,
                 keyType = "GID",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.01,
                 qvalueCutoff  = 0.05,
                 ont  = "BP")
    ego2 <- clusterProfiler::simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)
    assign(x = paste0(s, '.', c, '.ego2'), ego2)
    
    fig <- dotplot(ego2, showCategory=30)
    ggsave(filename = paste0(s,"_",c,".GO.pdf"), fig, width = 6, height = 12)
}

In [23]:
Hsap_compare_ego2 <- compareCluster(list(ohno = unique(c(Hsap.ohno$Ohno1, Hsap.ohno$Ohno2)), 
                                         SSDpara = unique(c(Hsap.SSDparalogs$Dup1, Hsap.SSDparalogs$Dup2))), 
                                    fun="enrichGO", OrgDb = org.Hs.eg.db, keyType = "ENSEMBL", 
                                    pAdjustMethod = "BH",
                                    pvalueCutoff = 0.01,
                                    qvalueCutoff  = 0.05,
                                    universe = Hsap_bg, ont = "BP")
Hsap_compare_ego2 <- clusterProfiler::simplify(Hsap_compare_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [40]:
# select GO terms for Fig2 f
options(enrichplot.colours = c("red","blue"))
fig <- dotplot(Hsap_compare_ego2, showCategory = 30)
ggsave(filename = "Hsap.ohnologs_vs_SSDparalogs.top30.pdf", fig, width = 7, height = 20)

selected <- c('synapse organization', 'cell fate commitment', 'axonogenesis',
              'neurotransmitter transport', 'signal release', 'regulation of nervous system development','sensory perception of smell', 
              'detection of chemical stimulus', 'humoral immune response', 
              'defense response to bacterium','response to chemokine','cell chemotaxis')
fig <- dotplot(Hsap_compare_ego2, showCategory = selected)
ggsave(filename = "Hsap.ohnologs_vs_SSDparalogs.selected.pdf", fig, width = 4.5, height = 7)

In [37]:
Mmus_compare_ego2 <- compareCluster(list(ohno = unique(c(Mmus.ohno$Ohno1, Mmus.ohno$Ohno2)),
                                         SSDpara = unique(c(Mmus.SSDparalogs$Dup1, Mmus.SSDparalogs$Dup2))),
                                    fun="enrichGO", OrgDb = org.Mm.eg.db, keyType = "SYMBOL",
                                    pAdjustMethod = "BH",
                                    pvalueCutoff = 0.01,
                                    qvalueCutoff  = 0.05,
                                    universe = Mmus_bg, ont = "BP")
Mmus_compare_ego2 <- clusterProfiler::simplify(Mmus_compare_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [56]:
fig <- dotplot(Mmus_compare_ego2, showCategory = 30)
ggsave(filename = "Mmus.ohnologs_vs_SSDparalogs.top30.pdf", fig, width = 7, height = 20)

selected <- c('regulation of synapse organization', 'cell fate commitment', 'axonogenesis','embryonic organ morphogenesis',
              'neurotransmitter transport', 'positive regulation of nervous system development',
              'humoral immune response', 
              'defense response to bacterium','response to chemokine','cell chemotaxis')
fig <- dotplot(Mmus_compare_ego2, showCategory = selected)
ggsave(filename = "Mmus.ohnologs_vs_SSDparalogs.selected.pdf", fig, width = 4.5, height = 7)

In [59]:
Pvit_compare_ego2 <- compareCluster(list(ohno = unique(c(Pvit.ohno$Ohno1, Pvit.ohno$Ohno2)),
                                         SSDpara = unique(c(Pvit.SSDparalogs$Dup1, Pvit.SSDparalogs$Dup2))),
                                    fun="enrichGO", OrgDb = org.Pvitticeps.eg.db, keyType = "GID",
                                    pAdjustMethod = "BH",
                                    pvalueCutoff = 0.01,
                                    qvalueCutoff  = 0.05,
                                    universe = Pvit_bg, ont = "BP")
Pvit_compare_ego2 <- clusterProfiler::simplify(Pvit_compare_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [72]:
fig <- dotplot(Pvit_compare_ego2, showCategory = 30)
ggsave(filename = "Pvit.ohnologs_vs_SSDparalogs.top30.pdf", fig, width = 7, height = 20)

selected <- c('regulation of synapse organization', 'axonogenesis','embryonic organ morphogenesis',
              'neurotransmitter transport','regulation of nervous system development',
              'humoral immune response', 
              'defense response to bacterium','response to chemokine','cell chemotaxis')
fig <- dotplot(Pvit_compare_ego2, showCategory = selected)
ggsave(filename = "Pvit.ohnologs_vs_SSDparalogs.selected.pdf", fig, width = 4.5, height = 7)

In [73]:
Pmar_compare_ego2 <- compareCluster(list(ohno = unique(c(Pmar.ohno$Ohno1, Pmar.ohno$Ohno2)),
                                         SSDpara = unique(c(Pmar.SSDparalogs$Dup1, Pmar.SSDparalogs$Dup2))),
                                    fun="enrichGO", OrgDb = org.Pmarinus.eg.db, keyType = "GID",
                                    pAdjustMethod = "BH",
                                    pvalueCutoff = 0.01,
                                    qvalueCutoff  = 0.05,
                                    universe = Pmar_bg, ont = "BP")
Pmar_compare_ego2 <- clusterProfiler::simplify(Pmar_compare_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [85]:
fig <- dotplot(Pmar_compare_ego2, showCategory = 30)
ggsave(filename = "Pmar.ohnologs_vs_SSDparalogs.top30.pdf", fig, width = 7, height = 20)

selected <- c('modulation of chemical synaptic transmission', 'negative regulation of cell differentiation',
              'sensory system development','forebrain development', 'embryonic organ development',
              'macrophage activation', 'involved in inflammatory', 'CUT metabolic process',
              'cytokine production involved in immune response','wound healing')
fig <- dotplot(Pmar_compare_ego2, showCategory = selected)
ggsave(filename = "Pmar.ohnologs_vs_SSDparalogs.selected.pdf", fig, width = 4.5, height = 7)

In [87]:
selected <- c('lipid catabolic process', 'organic acid biosynthetic process', 'detection of stimulus', 
             'hormone metabolic process','defense response to bacterium','response to toxic substance',
             'vascular process in circulatory system',
              'detection of mechanical stimulus','neurotransmitter metabolic process')
fig <- dotplot(Bflo.para.ego2, showCategory = selected)
ggsave(filename = "Bflo.paralogs.selected.pdf", fig, width = 4.5, height = 7)

In [86]:
save.image(file = "GO.enrichment.image")